# **Reel Good Movie Recommender System (Group 4)**

Name: Alireza Hatami

# **Preprocessing**

First, we import the necessary libraries and load the datasets:

In [193]:
import pandas as pd
import ast
import itertools
import requests

# Load Movies Metadata
movies = pd.read_csv('./data/movies_metadata.csv')

# Load Movie Credits
credits = pd.read_csv('./data/credits.csv')

# Load User Ratings for Movies
ratings = pd.read_csv('./data/ratings.csv')

C:\Users\alire\AppData\Local\Temp\ipykernel_1968\1859596736.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('./data/movies_metadata.csv')


# 1. Movies Dataset

In [20]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [21]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## 1.1. Handle Duplicate Values:

In [22]:
duplicate_count = movies.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 13


In [23]:
movies = movies.drop_duplicates()

## 1.2. Handle Missing Values:

To check for null values:

In [24]:
movies.isnull().sum()

adult                        0
belongs_to_collection    40959
budget                       0
genres                       0
homepage                 37673
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

We can save the data in a DataFrame called `df` to continue.

In [25]:
df = movies.dropna(subset=['id'])

There are some improper data types in the 'id' column. First, we attempt to drop the NaN values, and it appears to work, but when we run the next code, we encounter a ValueError because NaN values still exist. the reason is that when you use `pd.to_numeric` with `errors='coerce`, it attempts to convert the values in the `id` column to numeric. If any invalid values cannot be converted (e.g., strings or non-numeric characters), they will be coerced into NaN.

Therefore, we must drop the NaN one more time after the following code:

In [26]:
# Convert 'id' in movies_metadata to numeric, coercing errors to NaN
df['id'] = pd.to_numeric(df['id'], errors='coerce')

In [27]:
print(df['id'].dtype)

float64


In [28]:
df = df.dropna(subset=['id'])

In [29]:
df['id'].isnull().sum()

0

In [30]:
print(df['id'].dtype)

float64


In [31]:
df['id'] = df['id'].astype(int)

# 2. Credits Dataset

In [32]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [33]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


## 2.1 Handle Duplicate Values

In [34]:
duplicate_count3 = credits.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count3}")

Number of duplicate rows: 37


In [35]:
credits = credits.drop_duplicates()

## 2.2 Handle Missing Values

In [36]:
credits.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

Although the results indicate that there is no missing data, it is still possible for missing data to exist. This is because the entries in the `credits` file's columns consist of lists containing dictionaries with information about the cast members and crew. As a result, when we attempt to extract information, such as the actors in each movie, we may encounter empty dictionaries or fields. Therefore, further investigation is necessary.

We also need to ensure that the `credits` dataset includes only movies that are present in the `df` dataframe (movies' metadata). The reason is that we may need to combine our dataframes to consolidate all the information in one place.

In [39]:
# Filter the credits to include only 'df' dataframe
filtered_credits = credits[credits['id'].isin(df['id'])]

In [40]:
filtered_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45439 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45439 non-null  object
 1   crew    45439 non-null  object
 2   id      45439 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


# 3. Creating a Merged Dataset

In [41]:
merged = pd.merge(df, filtered_credits, on='id')

According to the result, all the movies in the metadata dataset are also available in credits.

The columns of the `merged` dataframe are:

In [42]:
print(merged.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')


Although the total of null values for genres is zero, there is still missing data for genres due to many fields containing empty lists. However, We can calculate the percentage of missing data for the rest of the columns:

In [58]:
# Calculate the percentage of missing values for each column
missing_percentage = (merged.isnull().sum() / len(merged)) * 100

# Filter only columns with missing values
missing_percentage = missing_percentage[missing_percentage > 0]

print(missing_percentage)

belongs_to_collection    90.117021
homepage                 82.893405
imdb_id                   0.037394
original_language         0.024196
overview                  2.098456
popularity                0.006599
poster_path               0.849061
production_companies      0.006599
production_countries      0.006599
release_date              0.191369
revenue                   0.006599
runtime                   0.571906
spoken_languages          0.006599
status                    0.184770
tagline                  55.100963
title                     0.006599
video                     0.006599
vote_average              0.006599
vote_count                0.006599
dtype: float64


## 3.1 Examining the Structure of the Data:

The 'genres' column contains string representations of lists of dictionaries. Each string in this column represents a JSON-like structure, where:
- Each dictionary contains key-value pairs such as 'id': 16 and 'name': 'Animation'.
- Multiple dictionaries are enclosed in a list.

In [10]:
# To display the first item in the 'genres' column
print("First item of 'genres' column:")
print(movies['genres'].iloc[0])

# To check the type of the first item in the 'genres' column
print("\nType of the items in 'genres' column:")
print(type(movies['genres'].iloc[0]))

First item of 'genres' column:
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

Type of the items in 'genres' column:
<class 'str'>


To properly extract the `genres` from this column, we need to:
- Convert the string representation of the list of dictionaries into an actual Python list of dictionaries.
- Extract the `name` field from each dictionary in the list.

We can use `ast` or Abstract Syntax Trees library to convert the string representation into an actual Python list. Later, we can define a function to convert strings into actual Python lists when we extract the genres. Inside the function, we can use a code like: `ast.literal_eval()`

In [ ]:
ast.literal_eval(genres)

Let's take a look at the `release_date` column and its format:

In [13]:
# Display the first item in the 'release_date' column
print("First item of 'release_date' column:")
print(movies['release_date'].iloc[0])

# Check the type of the first item in the 'release_date' column
print("\nType of the items in 'release_date' column:")
print(type(movies['release_date'].iloc[0]))

First item of 'release_date' column:
1995-10-30

Type of the items in 'release_date' column:
<class 'str'>


We can see that the date is stored as a string. To extract the year of the movie's release, we can simply take the first four characters of the string.

## 3.2 Extracting Genres

In [44]:
def extract_genres(genres):
    try:
        genres_list = ast.literal_eval(genres)
        return [genre['name'] for genre in genres_list]
    except (ValueError, TypeError):
        return []

In [45]:
merged['genres'] = merged['genres'].apply(extract_genres)

In [46]:
all_genres = list(itertools.chain.from_iterable(merged['genres']))

unique_genres = set(all_genres)
print(unique_genres)

{'Romance', 'Crime', 'Horror', 'War', 'Thriller', 'TV Movie', 'Comedy', 'Drama', 'Western', 'Adventure', 'Music', 'Foreign', 'Documentary', 'Fantasy', 'Family', 'Mystery', 'Animation', 'Action', 'History', 'Science Fiction'}


The genres provided by the website grouplens.org are listed below. However, based on the extraction of genres mentioned earlier, we can see that there are additional genres present in the data that are not included in the dataset from grouplens.org.

In [48]:
merged['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45457                 [Drama, Family]
45458                         [Drama]
45459       [Action, Drama, Thriller]
45460                              []
45461                              []
Name: genres, Length: 45462, dtype: object

Let's inspect the Non-List Entries: 
- we can identify the rows where the genres column does not contain lists.

In [49]:
non_list_entries = [i for i in merged['genres'] if type(i) != list]
non_list_entries

[]

In [50]:
unique_types = merged['genres'].apply(type).unique()
print(unique_types)

[<class 'list'>]


So, now we can be sure that our genre column is all the same type (list). However, we need to examine how many empty lists we have:

In [51]:
# To count the number of empty lists in the 'genres' column
number_of_empty_genre = sum(1 for x in merged['genres'] if x == [])
number_of_empty_genre

2441

This outlines the number of genres that are missing from our dataset.

### 3.2.1 Retrieving Missing Data for Genres:

In [52]:
# Filter rows where 'genres' is an empty list
empty_genres_df = merged[merged['genres'].apply(lambda x: x == [])]

In [54]:
# empty_genres_df.info()  # 2441 entries

In [55]:
empty_genres_df['imdb_id'].isnull().sum()

5

We have five movies that do not have IMDB IDs. As a result, we cannot use the API to retrieve information about these movies unless we extract their IMDB IDs from our `links.csv` file.

In [56]:
# Drop rows with null values in 'imdb_id' from empty_genres_df
empty_genres_df = empty_genres_df.dropna(subset=['imdb_id'])

#### 3.2.1.1 Using IMDB API to Retrieve Metadata (e.g. Genres) for Movies 

Since the IMDb API limits free data requests to 1000 daily, we need to divide our data into three parts:

In [95]:
# Divide the data into three parts
part_1 = empty_genres_df.iloc[0:1000]   # Rows 0 to 999
part_2 = empty_genres_df.iloc[1000:2000]  # Rows 1000 to 1999
part_3 = empty_genres_df.iloc[2000:2442]  # Rows 2000 to 2441

We can use a free API key to retrieve the metadata for movies for all the parts (The code below demonstrates how to utilize the API for `part_3`. Please make the necessary adjustments to retrieve the missing data for the other parts):

In [61]:
# List_of_imdbid_for_empty_genres_3 = part_3['imdb_id'].tolist()

In [62]:
# List_of_imdbid_for_empty_genres_3 

In [63]:
# import requests

# API_KEY = "********"
# BASE_URL = "http://www.omdbapi.com/"

# # Use the list directly since it's already 1000 entries
# metadata_list = []
# for imdb_id in List_of_imdbid_for_empty_genres_3:
#     params = {
#         'i': imdb_id,  # IMDb ID from the list
#         'apikey': API_KEY  # Your API key
#     }
#     response = requests.get(BASE_URL, params=params)
#     if response.status_code == 200:
#         metadata = response.json()
#         metadata_list.append(metadata)
#     else:
#         print(f"Failed to fetch data for IMDb ID: {imdb_id}")

# # Output the metadata_list or save it for further processing
# print(f"Fetched metadata for {len(metadata_list)} IMDb IDs.")

Fetched metadata for 441 IMDb IDs.


In [64]:
# temp_part3 = pd.DataFrame(metadata_list)
# temp_part3.to_csv('./data/imdb_metadata_part3.csv', index=False)

W can load the three CSV files, merge them into a single DataFrame, and save the combined dataset as `imdb_metadata.csv` using the following code:

In [102]:
# import pandas as pd

# part1 = pd.read_csv('./data/imdb_metadata_part1.csv')
# part2 = pd.read_csv('./data/imdb_metadata_part2.csv')
# part3 = pd.read_csv('./data/imdb_metadata_part3.csv')

# # Concatenate all parts into one DataFrame
# imdb_metadata = pd.concat([part1, part2, part3], ignore_index=True)

# imdb_metadata.to_csv('./data/imdb_metadata.csv', index=False)

# # Verify the result
# print(imdb_metadata.info())
# print(f"Total rows in merged dataset: {len(imdb_metadata)}")

Now, let's load the IMDb metadata into a DataFrame:

In [106]:
imdb_metadata = pd.read_csv('./data/imdb_metadata.csv')

In [113]:
# Check for null values in 'Genre' column

imdb_metadata['Genre'].isnull().sum()

12

In [114]:
# Drop rows where 'Genre' is null
imdb_metadata = imdb_metadata.dropna(subset=['Genre'])

In [116]:
# check for duplicates:
duplicate_count4 = imdb_metadata.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count4}")

Number of duplicate rows: 1


In [117]:
imdb_metadata = imdb_metadata.drop_duplicates()

We can use the `imdbID` column from `imdb_metadata` to fill in missing genres in `merged` based on the `imdb_id` column:

In [118]:
# Select relevant columns from imdb_metadata (only 'imdbID' and 'Genre')
imdb_genres = imdb_metadata[['imdbID', 'Genre']]

# Merge 'merged' with 'imdb_genres' using 'imdb_id' as the key
merged = merged.merge(imdb_genres, how='left', left_on='imdb_id', right_on='imdbID')

# Convert 'Genre' from a string into a list format to match existing 'genres' format
def convert_genres(genre_str):
    try:
        return genre_str.split(', ') if isinstance(genre_str, str) else []
    except:
        return []

merged['Genre'] = merged['Genre'].apply(convert_genres)

# Fill empty genres in 'genres' column with 'Genre' column from imdb_metadata
merged['genres'] = merged.apply(lambda row: row['Genre'] if row['genres'] == [] else row['genres'], axis=1)

# Drop the temporary 'Genre' and 'imdbID' columns
merged = merged.drop(columns=['Genre', 'imdbID'])

In [119]:
# Verify that empty genres are now filled
print(f"Remaining empty genres: {sum(1 for x in merged['genres'] if x == [])}")

Remaining empty genres: 16


We can drop the remaining empty genres:

In [121]:
# Drop rows where 'genres' is an empty list
merged = merged[merged['genres'].apply(lambda x: x != [])]

# Verify that there are no empty genres left
print(f"Remaining empty genres: {sum(1 for x in merged['genres'] if x == [])}")  # Should print 0

Remaining empty genres: 0


In [122]:
merged['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45457                 [Drama, Family]
45458                         [Drama]
45459       [Action, Drama, Thriller]
45460                         [Drama]
45461            [Documentary, Music]
Name: genres, Length: 45446, dtype: object

## 3.3 Extracting the Release Years of the Movies

In [123]:
merged['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45457           NaN
45458    2011-11-17
45459    2003-08-01
45460    1917-10-21
45461    2017-06-09
Name: release_date, Length: 45446, dtype: object

In [124]:
merged['release_date'].isnull().sum()

84

Drop rows where 'release_date' is missing:

In [125]:
merged = merged.dropna(subset=['release_date'])

We can extract the year from the `release_date` column. Since the `release_date` is in `YYYY-MM-DD` format, we can simply extract the first four characters:

In [126]:
# Convert 'release_date' to datetime format (to handle any inconsistencies)
merged['release_date'] = pd.to_datetime(merged['release_date'], errors='coerce')

# Extract the year
merged['year'] = merged['release_date'].dt.year

# Verify the results
print(merged[['release_date', 'year']].head())

  release_date  year
0   1995-10-30  1995
1   1995-12-15  1995
2   1995-12-22  1995
3   1995-12-22  1995
4   1995-02-10  1995


In [ ]:
# Drop the 'release_date' column
merged = merged.drop(columns=['release_date'])

### 3.3.1 Dropping Rows with Null Values for IMDB IDs

In [138]:
merged = merged.dropna(subset=['imdb_id'])

## 3.4 Extracting Actors' Names:

In the metadata file, the names of the actors are listed in the order of the main characters in the movie. Some movies feature many actors, while others have fewer. Since it's not feasible to identify the stars for the entire list of characters, only the first three actors have been extracted.

In [142]:
# Extract the first three actors
def get_first_three_actors(cast):
    try:
        cast_list = ast.literal_eval(cast)
        return [actor['name'] for actor in cast_list[:3]]
    except (ValueError, TypeError):
        return []

In [143]:
merged['first_three_actors'] = merged['cast'].apply(get_first_three_actors)

In [144]:
merged['first_three_actors']

0                      [Tom Hanks, Tim Allen, Don Rickles]
1           [Robin Williams, Jonathan Hyde, Kirsten Dunst]
2               [Walter Matthau, Jack Lemmon, Ann-Margret]
3        [Whitney Houston, Angela Bassett, Loretta Devine]
4               [Steve Martin, Diane Keaton, Martin Short]
                               ...                        
45456       [Patrick Bergin, Uma Thurman, David Morrissey]
45458           [Angel Aquino, Perry Dizon, Hazel Orencio]
45459         [Erika Eleniak, Adam Baldwin, Julie du Page]
45460    [Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...
45461                                                   []
Name: first_three_actors, Length: 45351, dtype: object

Now, let's explore how many empty lists we have:

In [145]:
empty_list_actors = sum([1 for i in merged['first_three_actors'] if i == []])
empty_list_actors

2340

One possible explanation for the missing data is that we have categories like "Documentary," which obviously do not feature any actors.

In [146]:
# Filter rows where 'first_three_actors' is an empty list and 'Documentary' is in genres
documentary_empty_actors = merged[(merged['first_three_actors'].apply(lambda x: x == [])) & (merged['genres'].apply(lambda x: 'Documentary' in x))]

len(documentary_empty_actors)

1444

Evidently, most movies that don't list actor names fall into the "Documentary" category.

### 3.4.1 Retrieving Missing Data for Actors:

We want to create a temporary DataFrame that includes only movies where the `first_three_actors` list is empty and the genre is not "Documentary", while also keeping their `imdb_id`:

In [147]:
# Filter movies where 'first_three_actors' is an empty list and they are NOT documentaries
temp_empty_actors_df = merged[(merged['first_three_actors'].apply(lambda x: x == [])) & (~merged['genres'].apply(lambda x: 'Documentary' in x))][['imdb_id', 'first_three_actors']]

# Display the first few rows of the temporary DataFrame
print(temp_empty_actors_df.head())

print(f"Number of movies with empty actors which don't belong to documentaries: {len(temp_empty_actors_df)}")

       imdb_id first_three_actors
137  tt0114618                 []
240  tt0113234                 []
620  tt0114305                 []
661  tt0114616                 []
679  tt0109751                 []
Number of movies with empty actors which don't belong to documentaries: 896


#### 3.4.1.1 Using IMDB API to Retrieve Metadata (Actors) for Movies 

We can now use IMDb's API to retrieve data for all the IMDb IDs we have.

In [148]:
# imdb_ids_to_fetch = temp_empty_actors_df['imdb_id'].tolist()

In [150]:
# import requests

# API_KEY = "********"
# BASE_URL = "http://www.omdbapi.com/"

# metadata_list = []
# for imdb_id in imdb_ids_to_fetch:
#     params = {
#         'i': imdb_id,
#         'apikey': API_KEY
#     }
#     response = requests.get(BASE_URL, params=params)
    
#     if response.status_code == 200:
#         metadata = response.json()
#         metadata_list.append(metadata)
#     else:
#         print(f"Failed to fetch data for IMDb ID: {imdb_id}")
# print(f"Fetched metadata for {len(metadata_list)} IMDb IDs.")

Fetched metadata for 896 IMDb IDs.


In [152]:
# actors_metadata_df = pd.DataFrame(metadata_list)

# actors_metadata_df.to_csv('./data/imdb_actors_metadata.csv', index=False)

We will load the retrieved metadata for the actors and check for any null values. We will also prepare to fill the empty lists for actors in the `merged` dataframe.

In [161]:
actors_metadata_df = pd.read_csv('./data/imdb_actors_metadata.csv')

In [162]:
actors_metadata_df['Actors'].isnull().sum()

194

In [163]:
# Drop rows where 'Actors' is null
actors_metadata_df = actors_metadata_df.dropna(subset=['Actors'])

To fill the empty `first_three_actors` lists in the `merged` DataFrame using the `actors_metadata_df` DataFrame (based on `imdb_id`), and extracting only the first three actors, we can do the following:

In [167]:
actors_metadata_df = actors_metadata_df[['imdbID', 'Actors']]

# Extract the first three actors from the 'Actors' column
def extract_first_three_actors(actors_str):
    try:
        return actors_str.split(', ')[:3] if isinstance(actors_str, str) else []
    except:
        return []

actors_metadata_df['first_three_actors'] = actors_metadata_df['Actors'].apply(extract_first_three_actors)

# Merge the actors_metadata_df with the merged DataFrame on 'imdb_id'
merged = merged.merge(actors_metadata_df[['imdbID', 'first_three_actors']], how='left', left_on='imdb_id', right_on='imdbID')

# Fill empty 'first_three_actors' lists in the merged DataFrame
merged['first_three_actors'] = merged.apply(
    lambda row: row['first_three_actors_y'] if row['first_three_actors_x'] == [] else row['first_three_actors_x'], axis=1
)

# Drop the temporary columns
merged = merged.drop(columns=['first_three_actors_x', 'first_three_actors_y', 'imdbID'])

**Note**: During this operation, all empty lists belonging to the `documentary` genre will be converted to null values. Therefore, we can check the number of null values.

In [179]:
merged['first_three_actors'].isnull().sum()

1638

In [181]:
# Count remaining null values in 'first_three_actors' that belong to documentaries
remaining_documentary_null_actors = merged[(merged['first_three_actors'].isnull()) & (merged['genres'].apply(lambda x: 'Documentary' in x))].shape[0]
print(f"Remaining null actor values in documentaries: {remaining_documentary_null_actors}")

Remaining null actor values in documentaries: 1444


Since we have 1,638 total null values in `first_three_actors`, and 1,444 of them belong to documentaries, we only want to drop the 194 rows that have null actors but are NOT documentaries.

In [182]:
# Drop only rows where 'first_three_actors' is null AND the movie is NOT a documentary
merged = merged[~((merged['first_three_actors'].isnull()) & (~merged['genres'].apply(lambda x: 'Documentary' in x)))]

# Verify the change
remaining_null_actors = merged['first_three_actors'].isnull().sum()
print(f"Remaining null actor values: {remaining_null_actors}")  # Should now be only 1,444 (documentaries)

Remaining null actor values: 1444


We can retain these null values for now and assign them a value such as "no names" later.

## 3.5 Extracting Directors' Names

In [184]:
# Extract the director's name
def get_director(crew):
    try:
        crew_list = ast.literal_eval(crew)
        for member in crew_list:
            if member['job'] == 'Director':
                return member['name']
        return None
    except (ValueError, TypeError):
        return None

In [185]:
merged['director'] = merged['crew'].apply(get_director)

C:\Users\alire\AppData\Local\Temp\ipykernel_13472\3850882758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['director'] = merged['crew'].apply(get_director)


In [186]:
merged['director']

0           John Lasseter
1            Joe Johnston
2           Howard Deutch
3         Forest Whitaker
4           Charles Shyer
               ...       
45346          John Irvin
45347            Lav Diaz
45348      Mark L. Lester
45349    Yakov Protazanov
45350       Daisy Asquith
Name: director, Length: 45157, dtype: object

In [188]:
print(f"A total of {merged['director'].isnull().sum()} directors' names are missing.")

A total of 828 directors' names are missing.


### 3.5.1 Using IMDB API to Retrieve Metadata (e.g. Directors) for Movies 

We can extract the IMDb IDs of movies that have missing director names and save them to a list using the following code:

In [189]:
# # Extract IMDb IDs where the 'director' column is null
# missing_directors_imdb_ids = merged[merged['director'].isnull()]['imdb_id'].tolist()

We can retrieve metadata for the list of IMDb IDs using the API:

In [190]:
# import requests

# API_KEY = "********"
# BASE_URL = "http://www.omdbapi.com/"

# metadata_list = []
# for imdb_id in missing_directors_imdb_ids:
#     params = {
#         'i': imdb_id,
#         'apikey': API_KEY
#     }
#     response = requests.get(BASE_URL, params=params)
    
#     if response.status_code == 200:
#         metadata = response.json()
#         metadata_list.append(metadata)
#     else:
#         print(f"Failed to fetch data for IMDb ID: {imdb_id}")
# print(f"Fetched metadata for {len(metadata_list)} IMDb IDs.")

Fetched metadata for 828 IMDb IDs.


In [191]:
# directors_metadata_df = pd.DataFrame(metadata_list)

# directors_metadata_df.to_csv('./data/imdb_directors_metadata.csv', index=False)

In [192]:
directors_metadata_df = pd.read_csv('./data/imdb_directors_metadata.csv')

Now, let's fill the null values in the `director` column of the `merged` DataFrame using the first name from the `Director` column in `directors_metadata_df`, matching on `imdb_id`:

In [ ]:
# Extract the first director's name (some movies may have multiple directors listed)
def get_first_director(director_str):
    try:
        return director_str.split(', ')[0] if isinstance(director_str, str) else None
    except:
        return None

directors_metadata_df['Director'] = directors_metadata_df['Director'].apply(get_first_director)

# Merge directors_metadata_df into merged DataFrame
merged = merged.merge(directors_metadata_df, how='left', left_on='imdb_id', right_on='imdbID')

# Fill missing director names in 'merged' with the first director from IMDb data
merged['director'] = merged.apply(
    lambda row: row['Director'] if pd.isnull(row['director']) else row['director'], axis=1
)

# Drop the temporary columns
merged = merged.drop(columns=['imdbID', 'Director'])

# # Verify the number of remaining null directors
# remaining_null_directors = merged['director'].isnull().sum()
# print(f"Remaining null director values: {remaining_null_directors}")

We will remove the remaining null values as they are not included in the metadata retrieved from IMDB.

In [196]:
# Drop rows where 'director' is null
merged = merged.dropna(subset=['director'])

In [197]:
merged['director'].isnull().sum()

0

## 3.6 ExtractingProduction Company

In [203]:
def get_production_company_names(companies):
    try:
        companies_list = ast.literal_eval(companies)  # Convert string to list
        return [company['name'] for company in companies_list]  # Extract names of all companies
    except (ValueError, TypeError):
        return []

In [204]:
merged['production_companies'] = merged['production_companies'].apply(get_production_company_names)

In [205]:
merged['production_companies']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45152    [Westdeutscher Rundfunk (WDR), Working Title F...
45153                                        [Sine Olivia]
45154                            [American World Pictures]
45155                                          [Yermoliev]
45156                                                   []
Name: production_companies, Length: 45059, dtype: object

In [206]:
# To count the number of empty lists in the 'production_companies' column
number_of_empty = sum(1 for x in merged['production_companies'] if x == [])
number_of_empty

11607

Certain features, such as "production_companies," may not be as important as genres, actors, and directors. If there is a significant amount of missing data and we do not have access to an API to retrieve this information, we may choose to disregard this content.

# 4. Ratings Dataset

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [194]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


### 4.1 Handling Duplicates:

In [195]:
duplicate_count2 = ratings.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count2}")

Number of duplicate rows: 0


### 4.2 Handling Missing Value:

In [196]:
ratings.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

# 5. Selecting Relevant Columns and Cleaning the Data:

Selecting Columns from Merged DataFrame:

In [199]:
# List of columns to drop (these are columns that were added while we were merging the data to fill up the missing fields)
columns_to_drop = [
    'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Writer', 'Actors', 'Plot',
    'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes',
    'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response', 'totalSeasons', 'Season', 'Episode', 'seriesID'
]

merged = merged.drop(columns=columns_to_drop)

In [200]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45059 entries, 0 to 45156
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45059 non-null  object 
 1   belongs_to_collection  4482 non-null   object 
 2   budget                 45059 non-null  object 
 3   genres                 45059 non-null  object 
 4   homepage               7721 non-null   object 
 5   id                     45059 non-null  int32  
 6   imdb_id                45059 non-null  object 
 7   original_language      45049 non-null  object 
 8   original_title         45059 non-null  object 
 9   overview               44125 non-null  object 
 10  popularity             45059 non-null  object 
 11  poster_path            44748 non-null  object 
 12  production_companies   45059 non-null  object 
 13  production_countries   45059 non-null  object 
 14  revenue                45059 non-null  float64
 15  ru

Before selecting the columns that are important to us, we can save the merged results in a CSV file:

In [202]:
# # Save the merged DataFrame to a CSV file
# merged.to_csv('./data/merged_df.csv', index=False)

#### Load Updated Merged DataFrame:

In [150]:
merged = pd.read_csv('./data/merged_df.csv')

In [151]:
# Define the final columns and their order
final_columns = ['id', 'title', 'year', 'genres', 'first_three_actors', 
                 'director', 'original_language', 'imdb_id']

# Create a new DataFrame with only these columns
merged_final = merged[final_columns]

In [152]:
merged_final.isnull().sum()

id                       0
title                    0
year                     0
genres                   0
first_three_actors    1422
director                 0
original_language       10
imdb_id                  0
dtype: int64

In [153]:
# Drop rows where 'original_language' is null
merged_final = merged_final.dropna(subset=['original_language'])

In [154]:
merged_final['first_three_actors'].isnull().sum()

1421

**Note:** These are all the movies that belong to the documentary genre. Therefore, all movies in the documentary genre can be assigned "no actors."

In [155]:
# Fill null values with "No Actor"
merged_final['first_three_actors'] = merged_final['first_three_actors'].fillna("['No Actor']")

In [156]:
merged_final['first_three_actors'].isnull().sum()

0

In [157]:
merged_final.duplicated().sum()

29

In [158]:
merged_final = merged_final.drop_duplicates()

Let's examine the structure of our data. It would be more effective to store the `genres` and `first_three_actors` as strings seperated by commas instead of as lists. Please note that these lists are represented as strings rather than as Python list types, as this is how the IMDb data and the data we used are formatted.

In [159]:
print(type(merged_final['genres'].iloc[0]))  # Checks the first row

<class 'str'>


In [160]:
merged_final.head()

,id,title,year,genres,first_three_actors,director,original_language,imdb_id
0,862,Toy Story,1995,"['Animation', 'Comedy', 'Family']","['Tom Hanks', 'Tim Allen', 'Don Rickles']",John Lasseter,en,tt0114709
1,8844,Jumanji,1995,"['Adventure', 'Fantasy', 'Family']","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",Joe Johnston,en,tt0113497
2,15602,Grumpier Old Men,1995,"['Romance', 'Comedy']","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']",Howard Deutch,en,tt0113228
3,31357,Waiting to Exhale,1995,"['Comedy', 'Drama', 'Romance']","['Whitney Houston', 'Angela Bassett', 'Loretta...",Forest Whitaker,en,tt0114885
4,11862,Father of the Bride Part II,1995,['Comedy'],"['Steve Martin', 'Diane Keaton', 'Martin Short']",Charles Shyer,en,tt0113041


In [161]:
# Convert string representations of lists back to actual lists, then join as comma-separated strings
merged_final['genres'] = merged_final['genres'].apply(lambda x: ', '.join(ast.literal_eval(x)) if isinstance(x, str) else x)
merged_final['first_three_actors'] = merged_final['first_three_actors'].apply(lambda x: ', '.join(ast.literal_eval(x)) if isinstance(x, str) else x)

We will replace the null values in the `first_three_actors`	column for documentary genre movies with "No Actor"

In [162]:
# Fill null values with "No Actor"
merged_final['first_three_actors'] = merged_final['first_three_actors'].fillna('No Actor')

In [163]:
merged_final['first_three_actors'].isnull().sum()

0

Our data has been cleaned and is free of any structural issues or missing values:

In [174]:
# # Save the final cleaned dataset
# merged_final.to_csv('./data/cleaned_data2.csv', index=False)

# 6. Load Clean Data:

In [180]:
clean_data = pd.read_csv('./data/cleaned_data2.csv')

In [181]:
clean_data.isnull().any()

id                    False
title                 False
year                  False
genres                False
first_three_actors    False
director              False
original_language     False
imdb_id               False
dtype: bool

In [182]:
clean_data.duplicated().any()

False

In [183]:
clean_data.head()

,id,title,year,genres,first_three_actors,director,original_language,imdb_id
0,862,Toy Story,1995,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Don Rickles",John Lasseter,en,tt0114709
1,8844,Jumanji,1995,"Adventure, Fantasy, Family","Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,en,tt0113497
2,15602,Grumpier Old Men,1995,"Romance, Comedy","Walter Matthau, Jack Lemmon, Ann-Margret",Howard Deutch,en,tt0113228
3,31357,Waiting to Exhale,1995,"Comedy, Drama, Romance","Whitney Houston, Angela Bassett, Loretta Devine",Forest Whitaker,en,tt0114885
4,11862,Father of the Bride Part II,1995,Comedy,"Steve Martin, Diane Keaton, Martin Short",Charles Shyer,en,tt0113041


In [185]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45020 entries, 0 to 45019
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  45020 non-null  int64 
 1   title               45020 non-null  object
 2   year                45020 non-null  int64 
 3   genres              45020 non-null  object
 4   first_three_actors  45020 non-null  object
 5   director            45020 non-null  object
 6   original_language   45020 non-null  object
 7   imdb_id             45020 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.7+ MB


Approximately 440 entries were lost during the preprocessing procedure.